# The mosquito question

More specifically, how many mosquitoes would it take, constantly biting you as much as they can, to overcome your body's natural regeneration and threaten your life?

We're going to approach this mathmatically using a number of assumptions.

When mosquitoes bite, they take a volume of blood that is made up of plasma and red blood cells (among other things), in a proportion that is roughly equal to the proportion of each in each human's blood (55% plasma, 45% red blood cells). Each of these blood components have different regeneration rates, so here, we calculate each separately to find out which would threaten a human life first.

First, let's import pandas.

In [188]:
import pandas as pd

## The human variables

Here are the values we're using based on assumptions about human beings. These are all based on basic research online and all volumes are in millilitres. unless otherwise specified. Because red cells regenerate at a different rate than your plasma, we calculate them both separately.

Blood regeneration data is sourced from the [American Red Cross website](https://www.redcrossblood.org/faq.html).

In [189]:
percent_blood_plasma = 0.55 ## Your blood is roughly 55% plasma. https://www.medicalnewstoday.com/articles/321122#shock-and-blood-loss
percent_blood_redcells = 0.45 ## The non-plasma part of blood is made up of mostly red blood cells. https://www.medicalnewstoday.com/articles/321122#shock-and-blood-loss

ml_per_pint = 473 # ml

redcell_daily_regen_lower = 1/672 # Pints per hour
redcell_daily_regen_lower = redcell_daily_regen_lower * ml_per_pint # Converted to ml.
redcell_daily_regen_lower = redcell_daily_regen_lower * 24

redcell_daily_regen_upper = 1/1008 # Pints per hour
redcell_daily_regen_upper = redcell_daily_regen_upper * ml_per_pint # Converted to ml.
redcell_daily_regen_upper = redcell_daily_regen_upper * 24

plasma_daily_regen = 1/24 # Pints per hour
plasma_daily_regen = plasma_daily_regen * ml_per_pint # Converted to ml.
plasma_daily_regen = plasma_daily_regen * 24

The amount of blood a human being has in its body is irrelevant here, as we're only interested in the tipping point where mosquitoes overwhelm your body's natural regeneration.

## The mosquito variables

In [190]:
daily_single_mosquito_drain = -0.005 / 3 # This is divided by three because a mosquito can take about 0.005 of blood every three days, at which point the digestion process is complete.

single_mosquito_drain_rate_plasma = daily_single_mosquito_drain * percent_blood_plasma
single_mosquito_drain_rate_redcells = daily_single_mosquito_drain * percent_blood_redcells

## Building the swarm

Let's build out our swarm of mosquitoes (a table where each row represents the net blood drain of a certain number of mosquitoes)

In [191]:
## An empty Pandas dataframe.
df = pd.DataFrame()

# A column from 1 to 2 million, representing how many mosquitoes.
df["mosquitoes"] = range(1, 5000000)

df["mosquito_drain_redcells"] = df['mosquitoes'] * single_mosquito_drain_rate_redcells 
df["mosquito_drain_plasma"] = df['mosquitoes'] * single_mosquito_drain_rate_plasma

df["net_redcell_regen_lower"] = redcell_daily_regen_lower + df["mosquito_drain_redcells"]
df["net_redcell_regen_upper"] = redcell_daily_regen_upper + df["mosquito_drain_redcells"]
df["net_plasma_regen"] = plasma_daily_regen + df["mosquito_drain_plasma"]

df = df.set_index("mosquitoes")

display(df)

,mosquito_drain_redcells,mosquito_drain_plasma,net_redcell_regen_lower,net_redcell_regen_upper,net_plasma_regen
mosquitoes,,,,,
1,-0.00075,-0.000917,16.892107,11.261155,472.999083
2,-0.00150,-0.001833,16.891357,11.260405,472.998167
3,-0.00225,-0.002750,16.890607,11.259655,472.997250
4,-0.00300,-0.003667,16.889857,11.258905,472.996333
5,-0.00375,-0.004583,16.889107,11.258155,472.995417
...,...,...,...,...,...
4999995,-3749.99625,-4583.328750,-3733.103393,-3738.734345,-4110.328750
4999996,-3749.99700,-4583.329667,-3733.104143,-3738.735095,-4110.329667
4999997,-3749.99775,-4583.330583,-3733.104893,-3738.735845,-4110.330583


In [192]:
danger_point_redcells_lower = df.index[df['net_redcell_regen_lower'] <= 0][0]
danger_point_redcells_upper = df.index[df['net_redcell_regen_upper'] <= 0][0]
danger_point_plasma = df.index[df['net_plasma_regen'] <= 0][0]

display("Red blood cell danger point (lower bound): " + danger_point_redcells_lower.astype(str) + " mosquitoes.")
display("Red blood cell danger point (upper bound): " + danger_point_redcells_upper.astype(str) + " mosquitoes.")
display("Blood plasma danger point: " + danger_point_plasma.astype(str) + " mosquitoes.")

'Red blood cell danger point (lower bound): 22524 mosquitoes.'

'Red blood cell danger point (upper bound): 15016 mosquitoes.'

'Blood plasma danger point: 516000 mosquitoes.'